# Random Forest Algorithm: Regression Models

#### Importación de librerías ⬇️

In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV

In [3]:
# Importar data de un csv
data = pd.read_csv('../los_data_gt.csv')

In [4]:
data.columns

Index(['dias_estancia', 'sexo', 'grupo_etnico', 'departamento', 'municipio',
       'causa_atencion', 'condicion_egreso', 'tratamiento_recibido',
       'edad_correcta', 'causa_categoria', 'edad_categoria', 'region'],
      dtype='object')

In [5]:
data.head()

,dias_estancia,sexo,grupo_etnico,departamento,municipio,causa_atencion,condicion_egreso,tratamiento_recibido,edad_correcta,causa_categoria,edad_categoria,region
0,1.0,Hombre,Ignorado,Guatemala,San Juan Sacatepéquez,P95X,Vivo,Médico,0.0,Certain conditions originating in the perinata...,Primera Infancia,Central
1,1.0,Mujer,Ignorado,Zacapa,Zacapa,P95X,Vivo,Médico,0.0,Certain conditions originating in the perinata...,Primera Infancia,Nororiente
2,2.0,Mujer,No indígena,Quetzaltenango,Quetzaltenango,P95X,Vivo,Médico,0.0,Certain conditions originating in the perinata...,Primera Infancia,Occidente
3,2.0,Mujer,No indígena,Quetzaltenango,Quetzaltenango,P95X,Vivo,Médico,0.0,Certain conditions originating in the perinata...,Primera Infancia,Occidente
4,2.0,Mujer,No indígena,Quetzaltenango,Quetzaltenango,P95X,Vivo,Médico,0.0,Certain conditions originating in the perinata...,Primera Infancia,Occidente


In [6]:
# Evaluación de variables
def change(variable, dictValue, df2):
    count = 1

    def changeToVariable(x):
        nonlocal count
        nonlocal dictValue
        if x in dictValue:
            return dictValue[x]
        else:
            dictValue[x] = count
            count += 1
            return dictValue[x]
        
    if not pd.api.types.is_numeric_dtype(df2[variable]):
        df2[variable] = df2[variable].apply(changeToVariable)


def chageNotNumericVars(df):
    dicVars = {}
    for col in df.columns:
        change(col, dicVars, df)
    return dicVars

In [7]:
def regressionRandomForest(X_train, y_train, X_test, y_test, deep=None):

    # Crear el modelo de Random Forest
    rf_model = RandomForestRegressor(random_state=42)

    rf_model.fit(X_train, y_train)

    # Predecir en el conjunto de prueba
    y_pred = rf_model.predict(X_test)

    # Calcular el error cuadrático medio
    mse = mean_squared_error(y_test, y_pred)
    print("Error cuadrático medio:", mse)

    # Calcular la raiz del error cuadratico medio
    rmse = root_mean_squared_error(y_test, y_pred)
    print("raiz del error cuadratico medio", rmse)

    # Calcular el coeficiente de determinación
    r2 = r2_score(y_test, y_pred)
    print("Coeficiente de determinación:", r2)

In [8]:
def findBestMaxDepth(X_train, y_train):
    # Crear el modelo de Random Forest
    rf_model = RandomForestRegressor(random_state=42)

    # Definir los hiperparámetros
    param_grid = {
        'max_depth': list(range(5, 25, 5))
    }

    # Crear el objeto de búsqueda en la grilla
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)

    # Realizar la búsqueda en la grilla
    grid_search.fit(X_train, y_train)

    # Obtener la mejor profundidad
    best_depth = grid_search.best_params_['max_depth']

    return best_depth

## Primer modelo:
pop condicion_egreso, usando el resto de las variables

In [9]:
dr = data.copy()

y = dr.pop('dias_estancia')
dr.pop('condicion_egreso')
x = dr
description = chageNotNumericVars(dr)

In [11]:
# Separar los conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento: (1782662, 10)
Tamaño del conjunto de prueba: (445666, 10)


In [11]:
regressionRandomForest(X_train, y_train, X_test, y_test)

Error cuadrático medio: 6.637661921527838
raiz del error cuadratico medio 2.576366030192107
Coeficiente de determinación: -0.00632898080276334


In [17]:
best_depth = findBestMaxDepth(X_train, y_train)
print(f"La mejor profundidad encontrada es: {best_depth}")

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] END ........................................max_depth=5; total time= 2.2min
[CV] END ........................................max_depth=5; total time= 2.2min
[CV] END ........................................max_depth=5; total time= 2.2min
[CV] END .......................................max_depth=10; total time= 4.0min
[CV] END .......................................max_depth=10; total time= 4.0min
[CV] END .......................................max_depth=10; total time= 4.0min
[CV] END .......................................max_depth=15; total time= 5.1min
[CV] END .......................................max_depth=15; total time= 5.1min
[CV] END .......................................max_depth=15; total time= 4.4min
[CV] END .......................................max_depth=20; total time= 5.0min
[CV] END .......................................max_depth=20; total time= 5.0min
[CV] END .......................................m

##### Test Data Set

In [23]:
# Crear el modelo de Random Forest con la mejor profundidad
rf_model = RandomForestRegressor(max_depth=best_depth, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio:", mse)

# Calcular la raiz del error cuadratico medio
rmse = root_mean_squared_error(y_test, y_pred)
print("raiz del error cuadratico medio", rmse)

# Calcular el coeficiente de determinación
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación:", r2)

Error cuadrático medio: 5.841146730337402
raiz del error cuadratico medio 2.4168464432680454
Coeficiente de determinación: 0.11442985416364448


#### Train Data Set

In [12]:
# Crear el modelo de Random Forest con la mejor profundidad
rf_model = RandomForestRegressor(max_depth=15, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Predecir en el conjunto de entrenamiento
y_train_pred = rf_model.predict(X_train)

# Calcular el error cuadrático medio
mse_train = mean_squared_error(y_train, y_train_pred)
print("Error cuadrático medio (entrenamiento):", mse_train)

# Calcular la raiz del error cuadratico medio
rmse_train = np.sqrt(mse_train)
print("Raiz del error cuadratico medio (entrenamiento):", rmse_train)

# Calcular el coeficiente de determinación
r2_train = r2_score(y_train, y_train_pred)
print("Coeficiente de determinación (entrenamiento):", r2_train)

Error cuadrático medio (entrenamiento): 5.180068824415249
Raiz del error cuadratico medio (entrenamiento): 2.275976455153974
Coeficiente de determinación (entrenamiento): 0.2085176362520409


### Primer modelo: Ajuste de hiperparámetros
pop en condicion_egreso, usando el resto de las variables

In [13]:
def regressionRandomForest2(X_train, y_train, X_test, y_test, best_params=None):
    rf_model = RandomForestRegressor(random_state=42, **(best_params if best_params else {}))
    rf_model.fit(X_train, y_train)
    y_pred = rf_model.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print("Error cuadrático medio:", mse)
    print("Raíz del error cuadrático medio:", rmse)
    print("Coeficiente de determinación:", r2)

In [29]:
def findBestParameters2(X_train, y_train):
    rf_model = RandomForestRegressor(random_state=42)

    param_grid = {
        'max_depth': [20, 10],
        'n_estimators': [200, 100],
        'min_samples_split': [2, 10],
        'min_samples_leaf': [1, 4],
        'max_features': ['sqrt']
    }

    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=3, verbose=2, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    return grid_search.best_params_

In [30]:
# Uso de GridSearchCV para optimizar los parámetros
best_params = findBestParameters2(X_train, y_train)
print(f"Los mejores parámetros encontrados son: {best_params}")

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Los mejores parámetros encontrados son: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}


#### Test Data Set

In [31]:
# Usar los mejores parámetros encontrados para entrenar y evaluar el modelo
regressionRandomForest2(X_train, y_train, X_test, y_test, best_params)

Error cuadrático medio: 5.775985625663179
Raíz del error cuadrático medio: 2.4033280312231993
Coeficiente de determinación: 0.12430886108355388


#### Train Data Set

In [13]:
# Crear el modelo de Random Forest con la mejor profundidad
rf_model = RandomForestRegressor(max_depth=20, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Predecir en el conjunto de entrenamiento
y_train_pred = rf_model.predict(X_train)

# Calcular el error cuadrático medio
mse_train = mean_squared_error(y_train, y_train_pred)
print("Error cuadrático medio (entrenamiento):", mse_train)

# Calcular la raiz del error cuadratico medio
rmse_train = np.sqrt(mse_train)
print("Raiz del error cuadratico medio (entrenamiento):", rmse_train)

# Calcular el coeficiente de determinación
r2_train = r2_score(y_train, y_train_pred)
print("Coeficiente de determinación (entrenamiento):", r2_train)

Error cuadrático medio (entrenamiento): 4.017334821777287
Raiz del error cuadratico medio (entrenamiento): 2.0043290203400455
Coeficiente de determinación (entrenamiento): 0.3861761747796456


## Segundo modelo
pop condicion_egreso, municipio, region. Filtrando el DataFrame para mantener solo las filas donde el departamento es Guatemala.

In [14]:
df_cp = data.copy()

df_cp.pop('condicion_egreso')
df_cp.pop('municipio')
df_cp.pop('region')

df_cp = df_cp[df_cp['departamento'] == 'Guatemala']
df_cp.pop('departamento')

freq = df_cp['dias_estancia'].value_counts()
df_cp = df_cp.loc[df_cp['dias_estancia'] <= 30]

uniques = freq[freq == 1].index
df_cp = df_cp.loc[~df_cp['dias_estancia'].isin(uniques)]

y = df_cp.pop('dias_estancia')
X = df_cp
description = chageNotNumericVars(X)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
best_depth = findBestMaxDepth(X_train, y_train)
print(f"La mejor profundidad encontrada es: {best_depth}")

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] END ........................................max_depth=5; total time=  46.4s
[CV] END ........................................max_depth=5; total time=  46.5s
[CV] END ........................................max_depth=5; total time=  46.7s
[CV] END .......................................max_depth=10; total time= 1.3min
[CV] END .......................................max_depth=10; total time= 1.3min
[CV] END .......................................max_depth=10; total time= 1.3min
[CV] END .......................................max_depth=15; total time= 1.6min
[CV] END .......................................max_depth=15; total time= 1.6min
[CV] END .......................................max_depth=15; total time= 1.4min
[CV] END .......................................max_depth=20; total time= 1.5min
[CV] END .......................................max_depth=20; total time= 1.5min
[CV] END .......................................m

#### Test Data Set

In [15]:
# Crear el modelo de Random Forest con la mejor profundidad
rf_model = RandomForestRegressor(max_depth=best_depth, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Predecir en el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio:", mse)

# Calcular la raiz del error cuadratico medio
rmse = root_mean_squared_error(y_test, y_pred)
print("raiz del error cuadratico medio", rmse)

# Calcular el coeficiente de determinación
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación:", r2)

Error cuadrático medio: 4.4064677438655435
raiz del error cuadratico medio 2.0991588181615852
Coeficiente de determinación: 0.13996794332514984


#### Train Data Set

In [16]:
# Crear el modelo de Random Forest con la mejor profundidad
rf_model = RandomForestRegressor(max_depth=best_depth, random_state=42)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Predecir en el conjunto de entrenamiento
y_train_pred = rf_model.predict(X_train)

# Calcular el error cuadrático medio
mse_train = mean_squared_error(y_train, y_train_pred)
print("Error cuadrático medio (entrenamiento):", mse_train)

# Calcular la raiz del error cuadratico medio
rmse_train = np.sqrt(mse_train)
print("Raiz del error cuadratico medio (entrenamiento):", rmse_train)

# Calcular el coeficiente de determinación
r2_train = r2_score(y_train, y_train_pred)
print("Coeficiente de determinación (entrenamiento):", r2_train)

Error cuadrático medio (entrenamiento): 3.926126832818633
Raiz del error cuadratico medio (entrenamiento): 1.9814456421559066
Coeficiente de determinación (entrenamiento): 0.2537315825239762


### Segundo modelo: Ajuste de hiperparámetros
pop en condicion_egreso, municipio, region. Filtrando el DataFrame para mantener solo las filas donde el departamento es Guatemala.

In [16]:
df_cp = data.copy()

df_cp.pop('condicion_egreso')
df_cp.pop('municipio')
df_cp.pop('region')

df_cp = df_cp[df_cp['departamento'] == 'Guatemala']
df_cp.pop('departamento')

freq = df_cp['dias_estancia'].value_counts()
df_cp = df_cp.loc[df_cp['dias_estancia'] <= 30]

uniques = freq[freq == 1].index
df_cp = df_cp.loc[~df_cp['dias_estancia'].isin(uniques)]

y = df_cp.pop('dias_estancia')
X = df_cp
description = chageNotNumericVars(X)

In [17]:
# Definición de hiperparámetros
param_grid = {
    'max_depth': [20, 15, 10, 5],
    'n_estimators': [200, 300, 150, 100]
}

# Modelo de Random Forest
rf = RandomForestRegressor(random_state=42)

# Crear el objeto de búsqueda por cuadrícula
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)

# Ajustar el modelo
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] END .....................max_depth=20, n_estimators=150; total time= 2.8min
[CV] END .....................max_depth=20, n_estimators=150; total time= 2.8min
[CV] END .....................max_depth=20, n_estimators=200; total time= 3.8min
[CV] END .....................max_depth=20, n_estimators=200; total time= 3.8min
[CV] END .....................max_depth=20, n_estimators=200; total time= 3.8min
[CV] END .....................max_depth=20, n_estimators=100; total time= 1.9min
[CV] END .....................max_depth=20, n_estimators=300; total time= 5.6min
[CV] END .....................max_depth=20, n_estimators=100; total time= 1.9min
[CV] END .....................max_depth=20, n_estimators=150; total time= 2.8min
[CV] END .....................max_depth=20, n_estimators=300; total time= 5.6min
[CV] END .....................max_depth=20, n_estimators=100; total time= 1.9min
[CV] END .....................max_depth=20, n_es

/opt/homebrew/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END .....................max_depth=15, n_estimators=150; total time= 2.5min
[CV] END .....................max_depth=15, n_estimators=100; total time= 1.6min
[CV] END .....................max_depth=15, n_estimators=100; total time= 1.6min
[CV] END .....................max_depth=15, n_estimators=100; total time= 1.6min
[CV] END .....................max_depth=15, n_estimators=300; total time= 4.9min
[CV] END .....................max_depth=15, n_estimators=300; total time= 4.9min
[CV] END .....................max_depth=15, n_estimators=300; total time= 4.9min
[CV] END .....................max_depth=10, n_estimators=200; total time= 2.6min
[CV] END .....................max_depth=10, n_estimators=200; total time= 2.5min
[CV] END .....................max_depth=10, n_estimators=200; total time= 2.6min
[CV] END .....................max_depth=10, n_estimators=150; total time= 1.9min
[CV] END .....................max_depth=10, n_estimators=150; total time= 1.9min
[CV] END ...................

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [20, 15, 10, 5],
                         'n_estimators': [200, 300, 150, 100]},
             scoring='neg_mean_squared_error', verbose=2)

### Test Data Set

In [18]:
# Ver los mejores hiperparámetros
print(grid_search.best_params_)

# Usar el mejor modelo
best_rf = grid_search.best_estimator_

# Predecir en el conjunto de prueba
y_pred = best_rf.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print("Error cuadrático medio:", mse)

# Calcular la raiz del error cuadratico medio
rmse = np.sqrt(mse)
print("Raiz del error cuadratico medio:", rmse)

# Calcular el coeficiente de determinación
r2 = r2_score(y_test, y_pred)
print("Coeficiente de determinación:", r2)

{'max_depth': 15, 'n_estimators': 300}
Error cuadrático medio: 4.403329642178853
Raiz del error cuadratico medio: 2.0984112185600927
Coeficiente de determinación: 0.1405804221187179


#### Train Data Set

In [19]:
# Ver los mejores hiperparámetros
print(grid_search.best_params_)

# Usar el mejor modelo
best_rf = grid_search.best_estimator_

# Predecir en el conjunto de prueba
y_pred = best_rf.predict(X_train)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_train, y_pred)
print("Error cuadrático medio:", mse)

# Calcular la raiz del error cuadratico medio
rmse = np.sqrt(mse)
print("Raiz del error cuadratico medio:", rmse)

# Calcular el coeficiente de determinación
r2 = r2_score(y_train, y_pred)
print("Coeficiente de determinación:", r2)

{'max_depth': 15, 'n_estimators': 300}
Error cuadrático medio: 3.9227737153277897
Raiz del error cuadratico medio: 1.980599332355686
Coeficiente de determinación: 0.25436893475177136
